# Image Captioning — BLIP Base on Flickr8k
**TL;DR:** Generate captions for Flickr8k samples with BLIP and compute a toy BLEU score.

**Models & Datasets:** [Salesforce/blip-image-captioning-base](https://huggingface.co/Salesforce/blip-image-captioning-base) (BSD-3-Clause), [Flickr8k (test subset)](https://huggingface.co/datasets/flickr8k) (Creative Commons)
**Run Profiles:** 🖥️ CPU | 🍎 Metal (Apple Silicon) | 🧪 Colab/T4 | ⚡ CUDA GPU
**Env (minimal):** python>=3.10, transformers, datasets, evaluate, accelerate (optional: peft, bitsandbytes, timm, diffusers)
**Colab:** [Open in Colab](https://colab.research.google.com/github/SSusantAchary/Hands-On-Huggingface-AI-Models/blob/main/notebooks/multimodal/captioning-blip-base_flickr8k_cpu-first.ipynb)

**Switches (edit in one place):**
- `device` = {"cpu","mps","cuda"}
- `precision` = {"fp32","fp16","bf16","int8","4bit"}  (apply only if supported)
- `context_len` / `image_res` / `batch_size`

**Footprint & Speed (fill after run):**
- Peak RAM: TODO
- Peak VRAM: TODO (if GPU)
- TTFB: TODO, Throughput: TODO, Load time: TODO

**Gotchas:** Long captions may need truncation to avoid tokenizer overflow ([Fixes & Tips](../fixes-and-tips/tokenizer-context-oom.md))



## Setup
Fetch a few Flickr8k images and prepare BLIP for captioning.


In [ ]:

import json
import os
import subprocess
import time
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from evaluate import load as load_metric
from transformers import pipeline

from notebooks._templates.measure import append_benchmark_row, measure_memory_speed

DEVICE_PREFERENCE = os.environ.get("HF_DEVICE", "cpu")
PRECISION = os.environ.get("HF_PRECISION", "fp32")

def resolve_device(preference: str = "cpu") -> str:
    if preference == "cuda" and torch.cuda.is_available():
        return "cuda:0"
    if preference == "mps" and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

DEVICE = resolve_device(DEVICE_PREFERENCE)
print(f"Using device={DEVICE}")

MODEL_ID = "Salesforce/blip-image-captioning-base"
OUTPUT_DIR = Path("outputs") / "blip-captioning"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

dataset = load_dataset("flickr8k", split="test[:4]")
images = dataset["image"]
references = [ann[0]["raw"] for ann in dataset["annotations"]]


## Generate captions


In [ ]:

torch.manual_seed(42)

load_start = time.perf_counter()
captioner = pipeline(
    "image-to-text",
    model=MODEL_ID,
    device=DEVICE,
    max_new_tokens=32,
)
load_time = time.perf_counter() - load_start

captions = [captioner(image)[0]["generated_text"] for image in images]

df = pd.DataFrame(
    {
        "caption": captions,
        "reference": references,
    }
)
display(df)

bleu = load_metric("bleu")
tokenised_preds = [caption.split() for caption in captions]
tokenised_refs = [[ref.split()] for ref in references]
bleu_score = bleu.compute(predictions=tokenised_preds, references=tokenised_refs)["bleu"]
print(f"Toy BLEU: {bleu_score:.3f}")

df.to_csv(OUTPUT_DIR / "captions.csv", index=False)


## Measurement


In [ ]:

def run_inference(recorder):
    outputs = [captioner(image)[0]["generated_text"] for image in images]
    if outputs:
        recorder.mark_first_token()
    recorder.add_items(sum(len(out.split()) for out in outputs))

metrics = measure_memory_speed(run_inference)

def fmt(value, digits=4):
    if value in (None, "", float("inf")):
        return ""
    return f"{value:.{digits}f}"

try:
    repo_commit = subprocess.check_output(["git", "rev-parse", "HEAD"], text=True).strip()
except Exception:  # noqa: BLE001
    repo_commit = ""

append_benchmark_row(
    task="image-captioning",
    model_id=MODEL_ID,
    dataset="flickr8k",
    sequence_or_image_res="varied",
    batch=str(len(images)),
    peak_ram_mb=fmt(metrics.get("peak_ram_mb"), 2),
    peak_vram_mb=fmt(metrics.get("peak_vram_mb"), 2),
    load_time_s=fmt(load_time, 2),
    ttfb_s=fmt(metrics.get("ttfb_s"), 3),
    tokens_per_s_or_images_per_s=fmt(metrics.get("throughput_per_s"), 3),
    precision=PRECISION,
    notebook_path="notebooks/multimodal/captioning-blip-base_flickr8k_cpu-first.ipynb",
    repo_commit=repo_commit,
)

with open(OUTPUT_DIR / "metrics.json", "w", encoding="utf-8") as fp:
    json.dump({"bleu": bleu_score, **metrics}, fp, indent=2)
metrics


## Results Summary
        - Observations: TODO
        - Metrics captured: see `benchmarks/matrix.csv`

        ## Next Steps
        - TODOs: fill in after benchmarking

        ## Repro
        - Seed: 42 (set in measurement cell)
        - Libraries: captured via `detect_env()`
        - Notebook path: `notebooks/multimodal/captioning-blip-base_flickr8k_cpu-first.ipynb`
        - Latest commit: populated automatically when appending benchmarks (if git available)
